# Clean polling place data for analysis

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

---

In [4]:
src = pd.read_csv(
    "output/polling_places_analysis.csv", dtype={"year": str, "fips": str}
)

In [5]:
src.head()

,fips,place,state,total_reg_voters,mail_ballots_sent,poll_place_elect_day,poll_place_early,total_votes_cast,year
0,0100100000,AUTAUGA COUNTY,AL,43695.0,1329.0,18.0,1.0,27813.0,2020
1,0100300000,BALDWIN COUNTY,AL,176668.0,11147.0,50.0,1.0,110214.0,2020
2,0100500000,BARBOUR COUNTY,AL,17850.0,726.0,16.0,1.0,10560.0,2020
3,0100700000,BIBB COUNTY,AL,15014.0,332.0,8.0,1.0,9630.0,2020
4,0100900000,BLOUNT COUNTY,AL,41927.0,1032.0,24.0,1.0,27665.0,2020


---

### Clean up and categorize place types

In [6]:
src["place_type"] = "Other"

In [7]:
src.loc[src["place"].str.contains("COUNTY", case=False), "place_type"] = "County"
src.loc[src["place"].str.contains("PARISH", case=False), "place_type"] = "County"
src.loc[src["place"].str.contains("CITY", case=False), "place_type"] = "City"
src.loc[src["place"].str.contains("TOWN", case=False), "place_type"] = "Town"
src.loc[src["place"].str.contains("VILLAGE", case=False), "place_type"] = "Village"

In [8]:
src.place_type.value_counts()

County     8536
Town       5470
Other      3398
City       2284
Village    1393
Name: place_type, dtype: int64

In [9]:
src["place_clean"] = src["place"].str.title().str.replace(" County", "", regex=False)

In [10]:
src.total_reg_voters.describe()

count     21063
unique    13917
top         0.0
freq        103
Name: total_reg_voters, dtype: object

---

In [11]:
missing_values = [
    "-999999.0",
    "-99.0",
    "-888888.0",
    "-88.0",
    "-999999: Data Not Available",
    "-888888: Not Applicable",
    "",
]
src = src.replace(missing_values, np.NaN)

In [17]:
src.sort_values("total_votes_cast", ascending=False).head()

,fips,place,state,total_reg_voters,mail_ballots_sent,poll_place_elect_day,poll_place_early,total_votes_cast,year,place_type,place_clean
177,0603700000,LOS ANGELES COUNTY,CA,7122542.0,6067822.0,811.0,811.0,4263059.0,2020,County,Los Angeles
6636,0603700000,LOS ANGELES COUNTY,CA,6754224.0,2412019.0,4535.0,6.0,3551506.0,2016,County,Los Angeles
13104,0603700000,LOS ANGELES COUNTY,CA,4758437.0,31159.0,4623.0,1.0,3136412.0,2012,County,Los Angeles
76,0401300000,MARICOPA COUNTY,AZ,2863040.0,2258790.0,175.0,91.0,2089563.0,2020,County,Maricopa
16513,7200000000,PUERTO RICO,PR,2402941.0,1711.0,7865.0,110.0,1878969.0,2012,Other,Puerto Rico


In [13]:
src[
    [
        "total_reg_voters",
        "mail_ballots_sent",
        "total_votes_cast",
        "poll_place_elect_day",
        "poll_place_early",
    ]
] = src[
    [
        "total_reg_voters",
        "mail_ballots_sent",
        "total_votes_cast",
        "poll_place_elect_day",
        "poll_place_early",
    ]
].astype(
    float
)

In [18]:
src[src["total_reg_voters"] > 100].isnull().sum()

In [45]:
df = src[src["total_reg_voters"] > 50000].copy()

---

### Polling place rate

Total number of election day polling places divided by the total population and multiplied by 100,000.

In [46]:
df["poll_place_100k"] = round(
    (df["poll_place_elect_day"] / df["total_reg_voters"]) * 100000, 2
)

In [48]:
df.sort_values("poll_place_100k", ascending=False).head(20)

,fips,place,state,total_reg_voters,mail_ballots_sent,poll_place_elect_day,poll_place_early,total_votes_cast,year,place_type,place_clean,poll_place_100k
16513,7200000000,PUERTO RICO,PR,2402941.0,1711.0,7865.0,110.0,1878969.0,2012,Other,Puerto Rico,327.31
16456,4202100000,CAMBRIA COUNTY,PA,87214.0,271.0,165.0,NaN,61326.0,2012,County,Cambria,189.19
10046,7200000000,PUERTO RICO,PR,2867558.0,1543.0,5406.0,110.0,1589991.0,2016,Other,Puerto Rico,188.52
16494,4209700000,NORTHUMBERLAND COUNTY,PA,54178.0,155.0,94.0,NaN,33464.0,2012,County,Northumberland,173.50
13913,1809500000,MADISON COUNTY,IN,94969.0,101.0,156.0,0.0,53066.0,2012,County,Madison,164.26
21056,5410700000,WOOD COUNTY,WV,55062.0,146.0,88.0,5.0,34563.0,2012,County,Wood,159.82
21043,5408100000,RALEIGH COUNTY,WV,55654.0,102.0,88.0,1.0,29389.0,2012,County,Raleigh,158.12
16496,4210100000,PHILADELPHIA COUNTY,PA,1098617.0,4391.0,1687.0,NaN,697624.0,2012,County,Philadelphia,153.56
9989,4202100000,CAMBRIA COUNTY,PA,86848.0,3083.0,133.0,NaN,64243.0,2016,County,Cambria,153.14
10029,4210100000,PHILADELPHIA COUNTY,PA,1102560.0,17058.0,1686.0,NaN,724380.0,2016,County,Philadelphia,152.92


---

### Export

In [20]:
df.to_csv("output/polling_places_analysis_clean.csv", index=False)